In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            database='supermarket',
            user='root',    # Replace with your MySQL username
            password='7402018010@0550' # Replace with your MySQL password
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None


In [ ]:
import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            user='root',  # Replace with your MySQL username
            password='7402018010@0550'  # Replace with your MySQL password
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

def initialize_db():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()

            # Create the database if it doesn't exist
            cursor.execute("CREATE DATABASE IF NOT EXISTS supermarket")

            # Use the database
            cursor.execute("USE supermarket")

            cursor.execute('''
            CREATE TABLE IF NOT EXISTS items (
                item_no INT AUTO_INCREMENT PRIMARY KEY,
                item_name VARCHAR(255) NOT NULL,
                category VARCHAR(255) NOT NULL,
                price DECIMAL(10, 2) NOT NULL,
                stocks_available INT NOT NULL
            )
            ''')

            cursor.execute('''
            CREATE TABLE IF NOT EXISTS bills (
                bill_id INT AUTO_INCREMENT PRIMARY KEY,
                customer_name VARCHAR(255) NOT NULL,
                item_no INT,
                quantity INT,
                total_price DECIMAL(10, 2),
                FOREIGN KEY (item_no) REFERENCES items(item_no)
            )
            ''')

            connection.commit()
            print("Tables created successfully")
        except Error as e:
            print("Error while creating tables", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def add_item():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("USE supermarket")
            item_name = input("Enter item name: ")
            category = input("Enter item category: ")
            price = float(input("Enter item price: "))
            stocks_available = int(input("Enter stocks available: "))

            cursor.execute('''
            INSERT INTO items (item_name, category, price, stocks_available)
            VALUES (%s, %s, %s, %s)
            ''', (item_name, category, price, stocks_available))
            connection.commit()
            print(f"Item '{item_name}' added successfully")
        except Error as e:
            print("Error while adding item", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def generate_bill():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("USE supermarket")
            customer_name = input("Enter customer name: ")
            items = []

            while True:
                item_no = int(input("Enter item number: "))
                quantity = int(input("Enter quantity: "))
                items.append((item_no, quantity))
                more_items = input("Do you want to add more items? (yes/no): ")
                if more_items.lower() != 'yes':
                    break

            total_price = 0
            for item_no, quantity in items:
                cursor.execute("SELECT price, stocks_available FROM items WHERE item_no = %s", (item_no,))
                result = cursor.fetchone()
                if not result or result[1] < quantity:
                    print(f"Insufficient stock for item number {item_no}")
                    return
                price = result[0]
                total_price += price * quantity
                cursor.execute("UPDATE items SET stocks_available = stocks_available - %s WHERE item_no = %s", (quantity, item_no))

            for item_no, quantity in items:
                cursor.execute('''
                INSERT INTO bills (customer_name, item_no, quantity, total_price)
                VALUES (%s, %s, %s, %s)
                ''', (customer_name, item_no, quantity, total_price))

            connection.commit()
            print(f"Bill generated successfully for {customer_name}. Total price: {total_price}")
        except Error as e:
            print("Error while generating bill", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def display_total_cost():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("USE supermarket")
            cursor.execute('''
            SELECT items.item_name, items.price, bills.quantity, (items.price * bills.quantity) as total_cost
            FROM bills
            JOIN items ON bills.item_no = items.item_no
            ''')
            purchases = cursor.fetchall()
            print(f"{'Item Name':<20} {'Price':<10} {'Quantity':<10} {'Total Cost':<10}")
            for purchase in purchases:
                print(f"{purchase[0]:<20} {purchase[1]:<10} {purchase[2]:<10} {purchase[3]:<10}")
        except Error as e:
            print("Error while fetching purchases", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def display_total_purchased_cost():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("USE supermarket")
            cursor.execute('''
            SELECT SUM(items.price * bills.quantity) as total_purchased_cost
            FROM bills
            JOIN items ON bills.item_no = items.item_no
            ''')
            total_cost = cursor.fetchone()[0]
            print(f"Total cost of all items purchased: {total_cost}")
        except Error as e:
            print("Error while fetching total purchased cost", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def view_items():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("USE supermarket")
            cursor.execute('''
            SELECT * FROM items
            ''')
            items = cursor.fetchall()
            print(f"{'Item No':<10} {'Item Name':<20} {'Category':<20} {'Price':<10} {'Stocks Available':<10}")
            for item in items:
                print(f"{item[0]:<10} {item[1]:<20} {item[2]:<20} {item[3]:<10} {item[4]:<10}")
        except Error as e:
            print("Error while fetching items", e)
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

def main():
    initialize_db()

    while True:
        print("\n1. Add Item")
        print("2. Generate Bill")
        print("3. Display Total Cost")
        print("4. View Items")
        print("5. Display Total Purchased Cost")
        print("6. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_item()
        elif choice == '2':
            generate_bill()
        elif choice == '3':
            display_total_cost()
        elif choice == '4':
            view_items()
        elif choice == '5':
            display_total_purchased_cost()
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Connected to MySQL database
Tables created successfully

1. Add Item
2. Generate Bill
3. Display Total Cost
4. View Items
5. Display Total Purchased Cost
6. Exit


Enter your choice:  1


Connected to MySQL database


Enter item name:  curd
Enter item category:  diary
Enter item price:  50
Enter stocks available:  50


Item 'curd' added successfully

1. Add Item
2. Generate Bill
3. Display Total Cost
4. View Items
5. Display Total Purchased Cost
6. Exit


...

Enter your choice:  3


Connected to MySQL database
Item Name            Price      Quantity   Total Cost
Apple                0.50       5          2.50      
Milk                 1.20       2          2.40      
Bread                1.50       1          1.50      
Apple                0.50       3          1.50      
Bread                1.50       4          6.00      
yoghurt              15.00      6          90.00     
cucumber             15.00      7          105.00    
baking soda          20.00      5          100.00    
Ice cream            30.00      10         300.00    
Bread                1.50       4          6.00      
Milk                 1.20       6          7.20      
Milk                 1.20       5          6.00      

1. Add Item
2. Generate Bill
3. Display Total Cost
4. View Items
5. Display Total Purchased Cost
6. Exit


Enter your choice:  2


Connected to MySQL database


Enter customer name:  alice
Enter item number:  6
